In [1]:
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

import numpy as np

import pydrake.all
from pydrake.all import (
    RigidTransform, RotationMatrix, AngleAxis, RollPitchYaw, InverseKinematics, MultibodyPlant, Parser,
    FindResourceOrThrow, Solve, PiecewisePolynomial, TrajectorySource, SceneGraph, DiagramBuilder,
    AddMultibodyPlantSceneGraph, LinearBushingRollPitchYaw, MathematicalProgram, AutoDiffXd, GenerateHtml, Role,
    LeafSystem, AbstractValue, PublishEvent, TriggerType, BasicVector, PiecewiseQuaternionSlerp,
    RandomGenerator, UniformlyRandomRotationMatrix, Simulator, FixedOffsetFrame
    )

from panda_station import *

In [2]:
"""
builder = DiagramBuilder()
station = PandaStation()


names_and_links = [(name, "base_link") for name in parse_tables(find_resource("directives/three_tables.yaml"))]
station.setup_from_file("directives/three_tables.yaml", names_and_links = names_and_links)
station.add_panda_with_hand()
plant, scene_graph = station.get_plant_and_scene_graph()

#station.add_model_from_file(find_resource("models/manipulands/sdf/foam_brick.sdf"), RigidTransform(RotationMatrix(), [0.6, 0, 0.2]))
station.add_model_from_file(find_resource("models/manipulands/sdf/foam_brick.sdf"), RigidTransform(RotationMatrix(), [0.8, 0, 0.2]),
                           main_body_name = "base_link", welded = True, name = "A")
station.add_model_from_file(find_resource("models/manipulands/sdf/dumbbell.sdf"), RigidTransform(),
                           main_body_name = "base_link", P = plant.GetFrameByName("panda_hand"), welded = True,
                            name = "B"
                           )


station.finalize()

builder.AddSystem(station)
meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,                                      
          zmq_url=zmq_url)#, role = Role.kProximity)# <- this commented part allows visualization of the collisions
meshcat.load()
diagram = builder.Build()
simulator = Simulator(diagram)
simulator_context = simulator.get_context()
station_context = station.GetMyContextFromRoot(simulator_context)
station.GetInputPort("panda_position").FixValue(station_context, np.array([np.pi/4, 0.1, 0, -1, 0, 1.5, 0]))
station.GetInputPort("hand_position").FixValue(station_context, [0.08])

meshcat.start_recording()
simulator.AdvanceTo(1.0)

print("PARSING STATION POSES")
print(parse_start_poses(station, station_context))
print("PARSING INITAL CONFIG")
print(parse_config(station, station_context))
print("PARSE TABLES")
print(parse_tables(find_resource("directives/three_tables.yaml")))

pose_fluents = [
    ("grasppose", "B", RigidTransform(RotationMatrix(), [0, 0, 0.3])),
    ("atpose", "A", RigidTransform(RotationMatrix(), [0.8, 0.2, 0.2]))
]
update_station(station, station_context, pose_fluents)

simulator_context = simulator.get_context()
plant_context = plant.GetMyContextFromRoot(simulator_context)
station_context = station.GetMyContextFromRoot(simulator_context)

simulator.AdvanceTo(2.0)
meshcat.stop_recording()
meshcat.publish_recording()
"""

'\nbuilder = DiagramBuilder()\nstation = PandaStation()\n\n\nnames_and_links = [(name, "base_link") for name in parse_tables(find_resource("directives/three_tables.yaml"))]\nstation.setup_from_file("directives/three_tables.yaml", names_and_links = names_and_links)\nstation.add_panda_with_hand()\nplant, scene_graph = station.get_plant_and_scene_graph()\n\n#station.add_model_from_file(find_resource("models/manipulands/sdf/foam_brick.sdf"), RigidTransform(RotationMatrix(), [0.6, 0, 0.2]))\nstation.add_model_from_file(find_resource("models/manipulands/sdf/foam_brick.sdf"), RigidTransform(RotationMatrix(), [0.8, 0, 0.2]),\n                           main_body_name = "base_link", welded = True, name = "A")\nstation.add_model_from_file(find_resource("models/manipulands/sdf/dumbbell.sdf"), RigidTransform(),\n                           main_body_name = "base_link", P = plant.GetFrameByName("panda_hand"), welded = True,\n                            name = "B"\n                           )\n\n\ns

In [3]:
p = ProblemInfo("/home/agrobenj/drake-tamp/experiments/pick-place-regions/problems/test_problem.yaml")
print(p)

directive: directives/three_tables.yaml
object: foam_brick
X_WO: RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [-0.0, 0.0, 1.0],
  ]),
  p=[0.6, 0.0, 0.2],
)
object: meat_can
X_WO: RigidTransform(
  R=RotationMatrix([
    [1.0, -0.0, 0.0],
    [0.0, 0.0007963267107332633, 0.9999996829318346],
    [-0.0, -0.9999996829318346, 0.0007963267107332633],
  ]),
  p=[0.5, 0.2, 0.2],
)
object: soup_can
X_WO: RigidTransform(
  R=RotationMatrix([
    [1.0, -0.0, 0.0],
    [0.0, 0.0007963267107332633, 0.9999996829318346],
    [-0.0, -0.9999996829318346, 0.0007963267107332633],
  ]),
  p=[0.5, -0.2, 0.2],
)



In [5]:
stations = p.make_all_stations()
print(list(stations.keys()))
builder = DiagramBuilder()
station = stations["foam_brick"]
builder.AddSystem(station)
plant, scene_graph = station.get_plant_and_scene_graph()
meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,                                      
          zmq_url=zmq_url)#, role = Role.kProximity)# <- this commented part allows visualization of the collisions
meshcat.load()

director = builder.AddSystem(TrajectoryDirector())
builder.Connect(station.GetOutputPort("panda_position_measured"), director.GetInputPort("panda_position"))
builder.Connect(station.GetOutputPort("hand_state_measured"), director.GetInputPort("hand_state"))
builder.Connect(director.GetOutputPort("panda_position_command"), station.GetInputPort("panda_position"))
builder.Connect(director.GetOutputPort("hand_position_command"), station.GetInputPort("hand_position"))

diagram = builder.Build()
simulator = Simulator(diagram)
simulator_context = simulator.get_context()
station_context = station.GetMyContextFromRoot(simulator_context)


q1 = np.array([0.0, 0.1, 0, -1.2, 0, 1.6, 0])
q2 = q1.copy()
q2[0] = np.pi/2
traj = find_traj(station, station_context, q1, q2)

meshcat.start_recording()
simulator.AdvanceTo(0.2)


samples = np.array([q1, q2])
t = np.array([0.2, 5.0])
panda_traj = PiecewisePolynomial.FirstOrderHold(t, samples.T)
qh1 = np.array([0])
qh2 = np.array([0.08])
samples = np.array([qh1, qh2])
hand_traj = PiecewisePolynomial.FirstOrderHold(t, samples.T)
director.add_panda_traj(panda_traj)
director.add_hand_traj(hand_traj)
simulator.AdvanceTo(5.0)
meshcat.stop_recording()
meshcat.publish_recording()

print("PARSING STATION POSES")
print(parse_start_poses(station, station_context))
print("PARSING INITAL CONFIG")
print(parse_config(station, station_context))
print("PARSING TABLES")
print(parse_tables(station.directive))

['main', 'move_free', 'foam_brick', 'meat_can', 'soup_can']
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.
[ 0.   0.1  0.  -1.2  0.   1.6  0. ]
[]
[ 1.57079633  0.1         0.         -1.2         0.          1.6         0.        ]
[]
[ 0.   0.1  0.  -1.2  0.   1.6  0. ]
[]
[ 1.57079633  0.1         0.         -1.2         0.          1.6         0.        ]
[]
[ -5.26533341e-03   1.60569187e-01  -1.77645523e-02  -1.18099158e+00
   1.12351255e-03   1.57271162e+00   1.06170970e-01]
[]
[-0.01053067  0.22113837 -0.0355291  -1.16198316  0.00224703  1.54542323
  0.21234194]
[]
[-0.015796    0.28170756 -0.05329366 -1.14297474  0.00337054  1.51813485
  0.31851291]
[]
[-0.02106133  0.34227675 -0.07105821 -1.12396632  0.00449405  1.49084647
  0.42468388]
[]
[-0.02632667  0.40284593 -0.08882276 -1.1049579   0.00561756  1.46355808
  0.53085485]
[]
[-0.031592    0

[ 1.86225579 -0.36506913  0.40305618 -1.0666649  -0.74130016  2.06112727
  0.05179894]
[]
[ 1.79748702 -0.26172043  0.31348814 -1.09629492 -0.57656679  1.95865454
  0.04028806]
[]
[ 1.73271825 -0.15837174  0.2239201  -1.12592495 -0.41183342  1.85618181
  0.02877719]
[]
[ 1.66794948 -0.05502304  0.13435206 -1.15555497 -0.24710005  1.75370909
  0.01726631]
[]
[ 1.60318071  0.04832565  0.04478402 -1.18518499 -0.08236668  1.65123636
  0.00575544]
[]
[ 0.96014926 -0.35408062  0.54679666 -1.00061679 -1.00018537  2.25397446
 -0.00908454]
[]
[ 1.24573062 -0.43248338  0.77220436 -0.90664602 -1.30308284  2.32967262
 -0.00673538]
[]
[ 1.3885213  -0.47168476  0.88490821 -0.85966063 -1.45453158  2.36752169
 -0.0055608 ]
[]
[ 1.10293994 -0.393282    0.65950051 -0.9536314  -1.15163411  2.29182354
 -0.00790996]
[]
[ 1.31712596 -0.45208407  0.82855628 -0.88315332 -1.37880721  2.34859716
 -0.00614809]
[]
[ 1.17433528 -0.41288269  0.71585243 -0.93013871 -1.22735848  2.31074808
 -0.00732267]
[]
[ 1.031544

PARSING STATION POSES
{'foam_brick': RigidTransform(
  R=RotationMatrix([
    [-0.0032172648617605393, 0.9999948155803083, 0.00013423603843463572],
    [0.9583424937579653, 0.0030449188914604044, 0.2856053100464151],
    [0.2856034206107723, 0.0010475120281701947, -0.958347321618826],
  ]),
  p=[9.226395135698605e-06, 0.6336654475334963, 0.5522668518604571],
), 'meat_can': RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 0.0007963267107332633, 0.9999996829318346],
    [0.0, -0.9999996829318346, 0.0007963267107332633],
  ]),
  p=[0.5, 0.2, 0.2],
), 'soup_can': RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 0.0007963267107332633, 0.9999996829318346],
    [0.0, -0.9999996829318346, 0.0007963267107332633],
  ]),
  p=[0.5, -0.2, 0.2],
)}
PARSING INITAL CONFIG
{'panda': array([  1.57080991e+00,   1.00202545e-01,   4.94137662e-05,
        -1.19988059e+00,  -1.56923279e-04,   1.58972111e+00,
        -3.15557566e-03])}
PARSING TABLES
['table', 'table_squa

In [ ]:
plant, scene_graph = station.get_plant_and_scene_graph()
plant_context = station.GetSubsystemContext(plant, station_context)
scene_graph_context = station.GetSubsystemContext(scene_graph, station_context)
panda = station.get_panda()
hand = station.get_hand()
query_output_port = scene_graph.GetOutputPort("query")

def is_colliding(q):
    plant.SetPositions(plant_context, panda, q)
    query_object = query_output_port.Eval(scene_graph_context)
    pairs = query_object.ComputePointPairPenetration()
    print('pairs:')
    for p in pairs:
        print(p.id_A, p.id_B)
    print("panda hand")
    ids = plant.GetBodyIndices(hand)
    for body_id in ids:
        body = plant.get_body(body_id)
        for col_id in plant.GetCollisionGeometriesForBody(body):
            print(col_id)
    print("panda")
    ids = plant.GetBodyIndices(panda)
    for body_id in ids:
        body = plant.get_body(body_id)
        for col_id in plant.GetCollisionGeometriesForBody(body):
            print(col_id)
    print("brick")
    brick = plant.GetModelInstanceByName("foam_brick")
    ids = plant.GetBodyIndices(brick)
    for body_id in ids:
        body = plant.get_body(body_id)
        for col_id in plant.GetCollisionGeometriesForBody(body):
            print(col_id)
    table = plant.GetModelInstanceByName("table")
    ids = plant.GetBodyIndices(table)
    for body_id in ids:
        body = plant.get_body(body_id)
        for col_id in plant.GetCollisionGeometriesForBody(body):
            print(col_id)
    return query_object.HasCollisions()

def isStateValid(state):
    q = state_to_q(state)
    return not is_colliding(q)

print(is_colliding(q1))